In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

In [5]:
df=pd.read_csv('C:/Users/Rishav/Desktop/machine learning/ratings.csv')

In [6]:
df.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 794.2 MB


In [8]:
df.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [9]:
df=df.drop(columns={'timestamp'})
df.head()

,userId,movieId,rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1221,5.0
4,1,1246,5.0


In [10]:
df[df.isnull().any(axis=1)]

,userId,movieId,rating


In [11]:
df=df.dropna()
df[df.isnull().any(axis=1)],df.head()

(Empty DataFrame
 Columns: [userId, movieId, rating]
 Index: [],
    userId  movieId  rating
 0       1      110     1.0
 1       1      147     4.5
 2       1      858     5.0
 3       1     1221     5.0
 4       1     1246     5.0)

In [12]:
df.head()

,userId,movieId,rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1221,5.0
4,1,1246,5.0


In [13]:
dd = df.sort_values(by='movieId')
dd.reset_index(drop=True, inplace=True)
dd.head()

,userId,movieId,rating
0,46848,1,4.0
1,226491,1,4.0
2,226490,1,5.0
3,121018,1,2.0
4,30092,1,3.0


In [14]:
dd['Count'] = dd.groupby('movieId')['movieId'].transform('count')
dd.head(66009)

,userId,movieId,rating,Count
0,46848,1,4.0,66008
1,226491,1,4.0,66008
2,226490,1,5.0,66008
3,121018,1,2.0,66008
4,30092,1,3.0,66008
...,...,...,...,...
66004,251876,1,4.5,66008
66005,67,1,5.0,66008
66006,10485,1,4.0,66008
66007,252374,1,5.0,66008


In [15]:
dd.shape

(26024289, 4)

In [16]:
popularity_threshold = 1000
dd= dd.query('Count >= @popularity_threshold')
dd.head()

,userId,movieId,rating,Count
0,46848,1,4.0,66008
1,226491,1,4.0,66008
2,226490,1,5.0,66008
3,121018,1,2.0,66008
4,30092,1,3.0,66008


In [17]:
dd.shape

(23346769, 4)

In [18]:
dd['movieId'].value_counts().sort_values(ascending=False)

movieId
356      91921
318      91082
296      87901
593      84078
2571     77960
         ...  
25771     1004
7017      1002
6650      1000
5105      1000
6763      1000
Name: count, Length: 3747, dtype: int64

In [19]:
dd.columns

Index(['userId', 'movieId', 'rating', 'Count'], dtype='object')

In [20]:
dd = dd.drop('Count',axis=1)
dd = pd.pivot_table(dd.head(200000), index='userId', columns='movieId', values='rating', aggfunc='mean')
dd.columns = [f'movie{col}' for col in dd.columns]
dd = dd.reset_index()
dd.head()

,userId,movie1,movie2,movie3,movie4,movie5,movie6,movie7,movie8,movie9,movie10
0,2,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
1,5,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN
2,8,4.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
3,9,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
dd=dd.fillna(0)
dd.head()

,userId,movie1,movie2,movie3,movie4,movie5,movie6,movie7,movie8,movie9,movie10
0,2,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
1,5,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
2,8,4.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
3,9,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:

def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0001, beta=0.02):

    Q = Q.T  
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0: 
                    eij = R[i][j] - np.dot(P[i, :], Q[:, j])
                    for k in range(K):
                         P[i][k] = np.clip(P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k]), -1e5, 1e5)
                         Q[k][j] = np.clip(Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j]), -1e5, 1e5)

    
        loss = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    loss += (R[i][j] - np.dot(P[i, :], Q[:, j])) ** 2
                    loss += beta * (np.sum(P[i, :] ** 2) + np.sum(Q[:, j] ** 2))

        if loss < 0.001:
            break

    return P, Q.T,loss

In [35]:

R = dd.head(100).values
# Parameters
N = R.shape[0]  
M = R.shape[1]  
K = 4 


P = np.random.rand(N, K)*0.1  
Q = np.random.rand(M, K)*0.1 

P_final, Q_final,loss = matrix_factorization(R, P, Q, K)
#print(P_final)
#print(Q_final)
R_approx = np.dot(P_final, Q_final.T)

print("Original matrix (R):")
print(R[:5, :5]) 
print("\nApproximated matrix (R_approx):")
print(R_approx[:5, :5]) 
print("\n\nApproximated loss (loss):")
print(loss) 

Original matrix (R):
[[ 2.   0.   0.   0.   0. ]
 [ 5.   0.   0.   0.   0. ]
 [ 8.   4.   0.   0.   0. ]
 [ 9.   4.5  0.   0.   0. ]
 [12.   4.   0.   0.   0. ]]

Approximated matrix (R_approx):
[[ 2.00032347  3.13003377  2.63715801  1.90700289  0.56330365]
 [ 5.00248979  4.1882426   3.21105612  2.31478056  0.67833524]
 [ 7.99609001  3.73121887  2.67935457  1.93394882  0.57447698]
 [ 8.99886337  4.43898584  3.30513705  2.3953911   0.72929069]
 [11.99770715  3.95277859  2.87415413  2.09497669  0.66033613]]


Approximated loss (loss):
1743.474538778608
